In [1]:
%load_ext autoreload
%autoreload 2

from transformer_lens import HookedTransformer
import torch as t
from huggingface_hub import login
from contextlib import contextmanager
import json
import torch.nn.functional as F
from tqdm.notebook import tqdm
import gc
from IPython.display import display, HTML
import html as html_escaper
import copy

from utils import *

# login("")
t.set_grad_enabled(False)

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
model_base = HookedTransformer.from_pretrained_no_processing("gemma-2-2b", device="cpu")
model_chat = HookedTransformer.from_pretrained_no_processing("gemma-2-2b-it", device="cuda")
model_hs = copy.deepcopy(model_chat)
clear_mem()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b-it into HookedTransformer


In [3]:
MAX_SEQ_LEN = 300
NUM_LINES = 2000
with open("2b_it_generations.jsonl", "r") as f:
    data = [json.loads(line)['conversation'] for i, line in enumerate(f) if i < NUM_LINES]
toks = model_base.to_tokens(data, prepend_bos=False)[:, :MAX_SEQ_LEN] 

In [10]:
clear_mem()

layer, head = 13, 0
kl_hs, kl_ablated = get_kl(model_base, model_hs, model_chat, toks[:100], layer, head, batch_size=16)

  0%|          | 0/6 [00:00<?, ?it/s]

In [12]:
top_vals_hs, top_inds_hs = matrix_topk(kl_hs, k=10)
top_vals_ablated, top_inds_ablated = matrix_topk(kl_ablated, k=10)
for (batch_idx, seq_idx) in top_inds_ablated:
    highlight_kl(model_base, model_hs, model_chat, toks[batch_idx, :seq_idx+30], layer, head, cscale=1, ablate=True)
    print("--"*100)

High KL: 0.7773 at token ' and'\n


,Token,p_HS,p_B
0,and,0.026,0.508
1,,0.002,0.003
2,**,0.416,0.352
3,images,0.004,0.002
4,the,0.551,0.133


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


High KL: 0.7773 at token ' **'\n


,Token,p_HS,p_B
0,and,0.026,0.508
1,,0.002,0.003
2,**,0.416,0.352
3,images,0.004,0.002
4,the,0.551,0.133


...

,Token,p_HS,p_B
0,strategic,0.003,0.037
1,business,0.093,0.305
2,technological,0.057,0.145
3,traditional,0.408,0.104
4,physical,0.107,0.027
5,purely,0.042,0.010


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


High KL: 0.7773 at token ' the'\n


,Token,p_HS,p_B
0,it,0.011,0.055
1,this,0.008,0.036
2,us,0.334,0.727
3,were,0.003,0.001
4,I,0.021,0.005
5,we,0.559,0.086


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
